# Anatomical receptive field analysis

#### Analysis and visualization pipeline:

1. Loading pre- and postsynaptic cell type curated data sets
2. Matching all presynaptic cell ids connecting to each postsynaptic cell
3. Analysis of spatial span of presynaptic inputs to unique postsynaptic cells
4. Plotting value distributions of (3)
5. Colorcoding (3) for every postsynaptic cell column on top of the medulla neuropile mesh
6. Heatmaps of (3) to explore the data and catch some extrem values

In [1]:
#Importing packages

import os
import pandas as pd
import numpy as np
from fafbseg import flywire
from caveclient import CAVEclient
client = CAVEclient('flywire_fafb_production')
from openpyxl import load_workbook

C:\Users\smolina\Anaconda3\envs\EM\lib\site-packages\blessed\terminal.py:186: UserWarning: Failed to setupterm(kind='xterm-color'): Could not find terminal xterm-color
  warnings.warn(msg)


### 1. Loading curated data sets

In [2]:
#Loading information  from excel files
PC_disc = 'D'
dataPath = f'{PC_disc}:\Connectomics-Data\FlyWire\Excels\drive-data-sets'
date = '20230912'
fileName_post = f'Tm9 proofreadings_{date}.xlsx'
filePath_post = os.path.join(dataPath,fileName_post)
fileName_pre = f'Dm12 proofreadings_{date}.xlsx'
filePath_pre = os.path.join(dataPath,fileName_pre)

#Loading file as DataFrame
post_df = pd.read_excel(filePath_post)
pre_df = pd.read_excel(filePath_pre)


In [3]:
# Filtering data

# Selecting the R optic lobe IDs
R_pre_df = pre_df[pre_df['hemisphere'] == 'R'].copy()
R_post_df = post_df[post_df['hemisphere'] == 'R'].copy()

# Selecting the backbone proofread IDs
R_post_df = R_post_df[R_post_df['backbone proofread (Y/N)'] == 'Y'].copy()

# Selecting presynaptic cells ids
pre_ids = R_pre_df['Updated_seg_id'].tolist()


In [4]:
# Visualize the final data sets
display(R_post_df.head()) # tail or head is fine
display(R_pre_df.tail()) # tail or head is fine

,XYZ-ME,symbol,XYZ-LO,quality comment,hemisphere,lab,author,name,seg_id,identified_in,...,patch_id,column_id,backbone proofread (Y/N),twigs proofread (Y/N),inputs_proofread (Y/N),dark_neurons (Y/N),bad_quality_rim (Y/N),healthy_L3 (Y/N),detached_lamina (Y/N),dorso-ventral
693,"190086.0,85837.0,3281.0",Tm9,"173170.0,78482.0,4592.0",NaN,R,Marion Silies,Sebastian Molina-Obando,"Tm9, Tm9_L, transmedullary neuron Tm9, FBbt_00...",720575940627187849,"Fischbach & Dittrich, 1989",...,NaN,NaN,Y,N,Y,NaN,NaN,Y,N,V
694,"195500.0,50706.0,3987.0",Tm9,"181888.0,53114.0,5025.0",NaN,R,Marion Silies,Sebastian Molina-Obando,"Tm9, Tm9_L, transmedullary neuron Tm9, FBbt_00...",720575940633362131,"Fischbach & Dittrich, 1989",...,NaN,NaN,Y,N,N,NaN,NaN,Y,N,D
695,"195267.0,48669.0,3884.0",Tm9,"181672.0,51656.0,5034.0",NaN,R,Marion Silies,Sebastian Molina-Obando,"Tm9, Tm9_L, transmedullary neuron Tm9, FBbt_00...",720575940612657522,"Fischbach & Dittrich, 1989",...,NaN,NaN,Y,N,Y,NaN,NaN,N,N,D
696,"199314.0,68960.0,3401.0",Tm9,"181158.0,66070.0,4835.0",NaN,R,Marion Silies,Sebastian Molina-Obando,"Tm9, Tm9_L, transmedullary neuron Tm9, FBbt_00...",720575940618481733,"Fischbach & Dittrich, 1989",...,NaN,NaN,Y,N,Y,NaN,NaN,Y,N,D
697,"202562.0,57002.0,4241.0",Tm9,"184741.0,57224.0,4775.0",NaN,R,Axel Borst Lab,Amalia Braun,"Tm9, Tm9_L, transmedullary neuron Tm9, Choline...",720575940629750844,"Fischbach & Dittrich, 1989",...,NaN,NaN,Y,N,N,NaN,NaN,Y,N,D


,XYZ-ME,symbol,XYZ-LO,quality comment,total_presynaptic_sites,hemisphere,lab,author,name,seg_id,...,optic_lobe_id,patch_id,column_id,backbone proofread (Y/N),twigs proofread (Y/N),inputs_proofread (Y/N),dark_neurons (Y/N),bad_quality_rim (Y/N),healthy_L3 (Y/N),detached_lamina (Y/N)
209,NaN,Dm12,NaN,NaN,NaN,R,NaN,NaN,NaN,720575940646209059,...,Dm12-R106,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN
210,NaN,Dm12,NaN,NaN,NaN,R,NaN,NaN,NaN,720575940623783437,...,Dm12-R107,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
211,NaN,Dm12,NaN,NaN,NaN,R,NaN,NaN,NaN,720575940634292119,...,Dm12-R108,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
212,NaN,Dm12,NaN,NaN,NaN,R,NaN,NaN,NaN,720575940633203852,...,Dm12-R109,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN
213,NaN,Dm12,NaN,NaN,NaN,R,NaN,NaN,NaN,720575940618595874,...,Dm12-R110,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


### 2. Matching all presynaptic cell ids connecting to each postsynaptic cell

In [5]:
# Updating presynaptic neurons if they are not up-to-date
if not np.unique(flywire.is_latest_root(pre_ids))[0]: #if not up-to-date
    update_df = flywire.update_ids(pre_ids, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
    up_to_date_pre_ids = update_df['new_id'].tolist()
    display(update_df[update_df['changed'] == True])
    if len(update_df[update_df['confidence']< 1]) == 0:
        print('Good to go and match ids in the next step')
    else:
        print('Consider updating your ids in the original data set')
else:
    up_to_date_pre_ids = pre_ids
    


Updating:   0%|          | 0/110 [00:00<?, ?it/s]

,old_id,new_id,confidence,changed
55,720575940622687220,720575940630032144,1,True


Good to go


In [9]:
## Getting a single postsynaptic side ID
curr_post_ls = R_post_df['Updated_seg_id'].tolist()

## Getting all synaptic connections

# Quick update
if not np.unique(flywire.is_latest_root(curr_post_ls))[0]: # if not up-to-date
    #Updating the IDs via Fafbseg
    update_df = flywire.update_ids(curr_post_ls, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
    curr_post_ls = update_df['new_id']
    display(update_df[update_df['confidence'] < 1])
    

# Fetch the neuron's inputs
curr_post_inputs = flywire.synapses.fetch_synapses(curr_post_ls, pre=False, post=True, attach=True, 
                                             min_score=50, clean=True, transmitters=False, 
                                             neuropils=True, batch_size=30, 
                                             dataset='production', progress=True,mat= "live")

# Filter connections just selected presynaptic cells
pre_post_match_df = curr_post_inputs[curr_post_inputs['pre'].isin(up_to_date_pre_ids)].copy()


Updating:   0%|          | 0/576 [00:00<?, ?it/s]

,old_id,new_id,confidence,changed


Fetching synapses:   0%|          | 0/20 [00:00<?, ?it/s]

In [11]:
pre_post_match_df

,pre,post,cleft_score,pre_x,pre_y,pre_z,post_x,post_y,post_z,id,neuropil
14,720575940624077005,720575940626579087,155,714384,331676,109080,714288,331716,109080,221908202,ME_L
76,720575940623345878,720575940659650945,142,778372,365500,169480,778256,365536,169480,5228051,ME_L
77,720575940623345878,720575940659650945,145,778368,365496,169520,778212,365528,169560,5228061,ME_L
84,720575940633432659,720575940630963275,143,767804,226416,125000,767796,226500,124960,77624730,ME_L
291,720575940632751928,720575940627342408,136,781400,352268,157200,781416,352136,157160,240226604,ME_L
...,...,...,...,...,...,...,...,...,...,...,...
137353,720575940614602415,720575940650399097,149,810828,249668,161000,810776,249572,161000,138972809,ME_L
137413,720575940614602415,720575940630703298,133,805624,246408,152080,805624,246576,152040,102702338,ME_L
137475,720575940633681235,720575940612052578,143,826920,271784,176200,827040,271776,176200,238710779,ME_L
137493,720575940614602415,720575940650399097,154,807088,250856,156840,807000,250888,156880,242635065,ME_L


### 3. Analysis of spatial span of presynaptic inputs to unique postsynaptic cells

In [13]:
# Aggregating data frame based on unique post and pre segment IDs
# While aggregating, counting the number of contacts for each pre-post pair

pre_post_counts = pre_post_match_df.groupby(['post', 'pre'])['pre'].count().reset_index(name='pre_count')

In [24]:
# Geeting the volum occupied by all pre cells connectecd to single post cell
curr_post = curr_post_ls[0]
display(pre_post_counts[pre_post_counts['post'] == curr_post])

curr_pre_ls = pre_post_counts[pre_post_counts['post'] == curr_post]['pre'].tolist()

# Fetch the inputs from presynaptic cells
curr_pre_inputs = flywire.synapses.fetch_synapses(curr_pre_ls, pre=False, post=True, attach=True, 
                                             min_score=50, clean=True, transmitters=False, 
                                             neuropils=True, batch_size=30, 
                                             dataset='production', progress=True,mat= "live")


,post,pre,pre_count
668,720575940627187849,720575940617202805,2
669,720575940627187849,720575940625057515,4


In [25]:
curr_pre_inputs

,pre,post,cleft_score,pre_x,pre_y,pre_z,post_x,post_y,post_z,id,neuropil
0,720575940632100683,720575940625057515,174,762816,336432,133800,762796,336516,133760,95735194,ME_L
2,720575940611674232,720575940625057515,154,764064,344788,137080,764016,344664,137080,225916664,ME_L
3,720575940611674232,720575940617202805,56,763912,345048,137040,763816,345008,137000,225916652,ME_L
4,720575940637565285,720575940617202805,145,746032,339164,132080,745856,339048,132120,202677519,ME_L
7,720575940647182371,720575940625057515,139,745172,348928,133800,745312,348920,133800,94914335,ME_L
...,...,...,...,...,...,...,...,...,...,...,...
2450,720575940626428615,720575940625057515,78,748120,348892,128720,748232,348920,128720,215574022,ME_L
2452,720575940626428615,720575940625057515,141,748260,348772,128840,748332,348860,128840,215574076,ME_L
2453,720575940626428615,720575940625057515,149,747032,350368,128840,747096,350516,128840,215574061,ME_L
2454,720575940628077640,720575940625057515,110,755576,350756,134400,755656,350864,134360,94126852,ME_L
